In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from collections import Counter

In [2]:
data = pd.read_csv('movie_bd_v5.csv')
data.head(5)

,imdb_id,budget,revenue,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
0,tt0369610,150000000,1513528810,Jurassic World,Chris Pratt|Bryce Dallas Howard|Irrfan Khan|Vi...,Colin Trevorrow,The park is open.,Twenty-two years after the events of Jurassic ...,124,Action|Adventure|Science Fiction|Thriller,Universal Studios|Amblin Entertainment|Legenda...,6/9/2015,6.5,2015
1,tt1392190,150000000,378436354,Mad Max: Fury Road,Tom Hardy|Charlize Theron|Hugh Keays-Byrne|Nic...,George Miller,What a Lovely Day.,An apocalyptic story set in the furthest reach...,120,Action|Adventure|Science Fiction|Thriller,Village Roadshow Pictures|Kennedy Miller Produ...,5/13/2015,7.1,2015
2,tt2908446,110000000,295238201,Insurgent,Shailene Woodley|Theo James|Kate Winslet|Ansel...,Robert Schwentke,One Choice Can Destroy You,Beatrice Prior must confront her inner demons ...,119,Adventure|Science Fiction|Thriller,Summit Entertainment|Mandeville Films|Red Wago...,3/18/2015,6.3,2015
3,tt2488496,200000000,2068178225,Star Wars: The Force Awakens,Harrison Ford|Mark Hamill|Carrie Fisher|Adam D...,J.J. Abrams,Every generation has a story.,Thirty years after defeating the Galactic Empi...,136,Action|Adventure|Science Fiction|Fantasy,Lucasfilm|Truenorth Productions|Bad Robot,12/15/2015,7.5,2015
4,tt2820852,190000000,1506249360,Furious 7,Vin Diesel|Paul Walker|Jason Statham|Michelle ...,James Wan,Vengeance Hits Home,Deckard Shaw seeks revenge against Dominic Tor...,137,Action|Crime|Thriller,Universal Pictures|Original Film|Media Rights ...,4/1/2015,7.3,2015


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1889 entries, 0 to 1888
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   imdb_id               1889 non-null   object 
 1   budget                1889 non-null   int64  
 2   revenue               1889 non-null   int64  
 3   original_title        1889 non-null   object 
 4   cast                  1889 non-null   object 
 5   director              1889 non-null   object 
 6   tagline               1889 non-null   object 
 7   overview              1889 non-null   object 
 8   runtime               1889 non-null   int64  
 9   genres                1889 non-null   object 
 10  production_companies  1889 non-null   object 
 11  release_date          1889 non-null   object 
 12  vote_average          1889 non-null   float64
 13  release_year          1889 non-null   int64  
dtypes: float64(1), int64(4), object(9)
memory usage: 140.3+ KB


# Предобработка

In [4]:
answers = {} # Создадим словарь для ответов

# The time given in the dataset is in string format.
# So we need to change this in datetime format
data.release_date = pd.to_datetime(data['release_date'])

# Добавим информацию о прибыльности фильма (колонка profit)
def profit_calc(rev, budg):
    return rev-budg
profit = pd.Series(list(map(profit_calc, data.revenue, data.budget)), 
                 name = 'profit')
data = pd.concat([data.loc[:, :'revenue'], profit, 
                  data.loc[:, 'original_title':]],
                  axis = 1)
data.sample(5)

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
1633,tt0375063,16000000,109502303,93502303,Sideways,Paul Giamatti|Thomas Haden Church|Virginia Mad...,Alexander Payne,In search of wine. In search of women. In sear...,Two middle-aged men embark on a spiritual jour...,126,Comedy|Drama|Romance,Fox Searchlight Pictures|Michael London Produc...,2004-10-22,6.6,2004
686,tt0406759,12000000,56309766,44309766,The Eye,Jessica Alba|Alessandro Nivola|Parker Posey|Ra...,David Moreau|Xavier Palud,How can you believe your eyes when they're not...,Violinist Sydney Wells was accidentally blinde...,98,Drama|Horror|Thriller,Paramount Vantage|VN Productions,2008-02-01,5.6,2008
94,tt2473510,10000000,78099553,68099553,Paranormal Activity: The Ghost Dimension,Chris J. Murray|Brit Shaw|Ivy George|Lauren Fr...,Gregory Plotkin,You can't save them. All you can do is watch.,"Using a special camera that can see spirits, a...",88,Horror|Thriller,Paramount Pictures|Blumhouse Productions,2015-10-21,5.0,2015
80,tt2752772,10000000,52882018,42882018,Sinister 2,Shannyn Sossamon|James Ransone|Tate Ellington|...,Ciaran Foy,"Be careful, children at play.",A young mother and her twin sons move into a r...,97,Horror,Alliance Films|Blumhouse Productions|IM Global...,2015-08-19,5.5,2015
920,tt0253867,43000000,68696770,25696770,The Sweetest Thing,Cameron Diaz|Christina Applegate|Thomas Jane|S...,Roger Kumble,A romantic comedy without the sugar.,Christina's love life is stuck in neutral. Aft...,84,Romance|Comedy,Konrad Pictures,2002-04-12,5.4,2002


# 1. У какого фильма из списка самый большой бюджет?

Использовать варианты ответов в коде решения запрещено.    
Вы думаете и в жизни у вас будут варианты ответов?)

In [5]:
# В словарь вставляем номер вопроса и ваш ответ на него
# Пример: 
answers['1'] = '2. Spider-Man 3 (tt0413300)'
# Запишите свой вариант ответа
# Если ответили верно, можете добавить комментарий со значком "+"

answers['1'] = 'Pirates of the Caribbean: On Stranger Tides (tt1298650)'

In [6]:
# Тут пишем ваш код для решения данного вопроса:
data[data.budget == data.budget.max()]

,imdb_id,budget,revenue,profit,original_title,cast,director,tagline,overview,runtime,genres,production_companies,release_date,vote_average,release_year
723,tt1298650,380000000,1021683000,641683000,Pirates of the Caribbean: On Stranger Tides,Johnny Depp|PenÃ©lope Cruz|Geoffrey Rush|Ian M...,Rob Marshall,Live Forever Or Die Trying.,Captain Jack Sparrow crosses paths with a woma...,136,Adventure|Action|Fantasy,Walt Disney Pictures|Jerry Bruckheimer Films|M...,2011-05-11,6.3,2011


ВАРИАНТ 2

In [7]:
# Можно добавлять разные варианты решения

# 2. Какой из фильмов самый длительный (в минутах)?

In [8]:
# Длительность фильма указана в колонке runtime
title_info = data[data.runtime == data.runtime.max()].original_title
imdb_id_info = data[data.runtime == data.runtime.max()].imdb_id

# Приводим данные к строке
title_2 = title_info.to_string(index = False)[1:]
imdb_id_2 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_2
answer_2 = '{title} ({imdb_id})'.format(title=title_2, 
                                        imdb_id=imdb_id_2)

In [9]:
# добавляем ответ в словарь
answers['2'] = answer_2
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)'}

# 3. Какой из фильмов самый короткий (в минутах)?





In [10]:
# Длительность фильма указана в колонке runtime
title_info = data[data.runtime == data.runtime.min()].original_title
imdb_id_info = data[data.runtime == data.runtime.min()].imdb_id

# Приводим данные к строке
title_3 = title_info.to_string(index = False)[1:]
imdb_id_3 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_3
answer_3 = '{title} ({imdb_id})'.format(title=title_3, 
                                        imdb_id=imdb_id_3)

In [19]:
answers['3'] = answer_3
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)'}

# 4. Какова средняя длительность фильмов?


In [23]:
# Не мудрувствуя лукаво
answers['4'] = round(data.runtime.mean())
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110}

# 5. Каково медианное значение длительности фильмов? 

In [24]:
answers['5'] = round(data.runtime.median())

# 6. Какой самый прибыльный фильм?
#### Внимание! Здесь и далее под «прибылью» или «убытками» понимается разность между сборами и бюджетом фильма. (прибыль = сборы - бюджет) в нашем датасете это будет (profit = revenue - budget) 

In [25]:
# Бюджет фильма указан в колонке profit
title_info = data[data.profit == data.profit.max()].original_title
imdb_id_info = data[data.profit == data.profit.max()].imdb_id

# Приводим данные к строке
title_6 = title_info.to_string(index = False)[1:]
imdb_id_6 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_6
answer_6 = '{title} ({imdb_id})'.format(title=title_6, 
                                        imdb_id=imdb_id_6)
answers['6'] = answer_6
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)'}

# 7. Какой фильм самый убыточный? 

In [26]:
# Бюджет фильма указан в колонке profit
title_info = data[data.profit == data.profit.min()].original_title
imdb_id_info = data[data.profit == data.profit.min()].imdb_id

# Приводим данные к строке
title_7 = title_info.to_string(index = False)[1:]
imdb_id_7 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_6
answer_7 = '{title} ({imdb_id})'.format(title=title_7, 
                                        imdb_id=imdb_id_7)
answers['7'] = answer_7
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)'}

# 8. У скольких фильмов из датасета объем сборов оказался выше бюджета?

In [34]:
answer_8 = data[data.revenue > data.budget].imdb_id.nunique()
answers['8'] = answer_8

# 9. Какой фильм оказался самым кассовым в 2008 году?

In [49]:
# Опрделяем фильмы 2008 года
movies = data[data.release_year == 2008]

# "Кассовость" фильма указана в колонке revenue
title_info = movies[movies.revenue 
                      == movies.revenue.max()].original_title
imdb_id_info = movies[movies.revenue 
                      == movies.revenue.max()].imdb_id

title_9 = title_info.to_string(index = False)[1:]
imdb_id_9 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_9
answer_9 = '{title} ({imdb_id})'.format(title=title_9, 
                                        imdb_id=imdb_id_9)
answers['9'] = answer_9
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)'}

# 10. Самый убыточный фильм за период с 2012 по 2014 г. (включительно)?


In [51]:
# Опрделяем фильмы 20012 - 2014 годов
movies = data[(data.release_year >= 2012) &
              (data.release_year <= 2014)]

title_info = movies[movies.profit 
                      == movies.profit.min()].original_title
imdb_id_info = movies[movies.profit 
                      == movies.profit.min()].imdb_id

title_10 = title_info.to_string(index = False)[1:]
imdb_id_10 = imdb_id_info.to_string(index = False)[1:]

# Ответ передаем в переменную answer_9
answer_10 = '{title} ({imdb_id})'.format(title=title_10, 
                                        imdb_id=imdb_id_10)
answers['10'] = answer_10
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)',
 '4': 110,
 '5': 107,
 '6': 'Avatar (tt0499549)',
 '7': 'The Lone Ranger (tt1210819)',
 '8': 1478,
 '9': 'The Dark Knight (tt0468569)',
 '10': 'The Lone Ranger (tt1210819)'}

# 11. Какого жанра фильмов больше всего?

In [15]:
# эту задачу тоже можно решать разными подходами, попробуй реализовать разные варианты
# если будешь добавлять функцию - выноси ее в предобработку что в начале

ВАРИАНТ 2

# 12. Фильмы какого жанра чаще всего становятся прибыльными? 

# 13. У какого режиссера самые большие суммарные кассовые сборы?

# 14. Какой режисер снял больше всего фильмов в стиле Action?

# 15. Фильмы с каким актером принесли самые высокие кассовые сборы в 2012 году? 

# 16. Какой актер снялся в большем количестве высокобюджетных фильмов?

# 17. В фильмах какого жанра больше всего снимался Nicolas Cage? 

# 18. Самый убыточный фильм от Paramount Pictures

# 19. Какой год стал самым успешным по суммарным кассовым сборам?

# 20. Какой самый прибыльный год для студии Warner Bros?

# 21. В каком месяце за все годы суммарно вышло больше всего фильмов?

# 22. Сколько суммарно вышло фильмов летом? (за июнь, июль, август)

# 23. Для какого режиссера зима – самое продуктивное время года? 

# 24. Какая студия дает самые длинные названия своим фильмам по количеству символов?

# 25. Описание фильмов какой студии в среднем самые длинные по количеству слов?

# 26. Какие фильмы входят в 1 процент лучших по рейтингу? 
по vote_average

# 27. Какие актеры чаще всего снимаются в одном фильме вместе?


ВАРИАНТ 2

# Submission

In [16]:
# в конце можно посмотреть свои ответы к каждому вопросу
answers

{'1': 'Pirates of the Caribbean: On Stranger Tides (tt1298650)',
 '2': 'Gods and Generals (tt0279111)',
 '3': 'Winnie the Pooh (tt1449283)'}

In [17]:
# и убедиться что ни чего не пропустил)
len(answers)

3